In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast


In [ ]:
TWO_WORD_NAMES={'Aston martin':'AstonMartin','Alfa Romeo':'AlfaRomeo',
                'Great Wall':'GreatWall','Ineos Grenadier':'IneosGrenadier',
                'Land Rover':'LandRover','Link & Co':'Link&Co','SH auto':'SHauto',
                'SIN CARS':'SINCARS','Ssang yong':'Ssangyong'}

In [ ]:
#df=pd.concat([pd.read_csv('scraping_results/out_more.csv',index_col=0),pd.read_csv('scraping_results/out.csv',index_col=0)],ignore_index=True).drop_duplicates(subset=['URL'],keep='first')
df=pd.read_csv("scraping_results/latest.csv",index_col=0)
df

In [ ]:
#df.to_csv('out_merged.csv')

In [ ]:
def get_brand(x):
    res=x.split(' ')
    if (res[0]+' '+res[1])in TWO_WORD_NAMES:
        return TWO_WORD_NAMES[res[0]+' '+res[1]]
    return res[0]
def get_model(x):
    res=x.split(' ')
    if (res[0]+' '+res[1])in TWO_WORD_NAMES:
        return " ".join(res[2:])
    return " ".join(res[1:])

df['Brand']=df['CarModel'].map(get_brand).str.strip()
df['Model']=df['CarModel'].map(get_model).str.strip()
df['Brand'].value_counts()


In [ ]:
df=df.join(df['Дата на производствоTech'].str.split(' ',expand=True)).rename(columns={0:'Month',1:'Year'})
months_bulgarian = {
    "януари": 1,
    "февруари": 2,
    "март": 3,
    "април": 4,
    "май": 5,
    "юни": 6,
    "юли": 7,
    "август": 8,
    "септември": 9,
    "октомври": 10,
    "ноември": 11,
    "декември": 12
}
df['Month']=df['Month'].map(months_bulgarian)

In [ ]:
df['CarPrice']=df['CarPrice'].str.replace('( лв.| )','',regex=True).astype('float64')
df['МощностTech']=df['МощностTech'].str.replace(' к.с.','').str.strip().astype('float64')
df['Пробег [км]Tech']=df['Пробег [км]Tech'].str.replace(' км','').str.strip().astype('float64')
df['Пробег с едно зареждане (WLTP) [км]']=df['Пробег с едно зареждане (WLTP) [км]'].str.replace(' км.','').str.strip().astype('float64')
df['Капацитет на батерията [kWh]']=df['Капацитет на батерията [kWh]'].str.replace(' kWh.','').str.strip().astype('float64')

In [ ]:
df=df.join(df['Location'].str.split(',',expand=True).rename(columns={0:'Region',1:'City'}))
df['Region']=df['Region'].str.replace('обл. ','',).str.strip()
df['City']=df['City'].str.replace('гр. ','',).str.strip()

In [ ]:
def flatten_2d(arr):
    res=[]
    for i in arr:
        for j in i:
            res.append(j)
    return res

In [ ]:
for col in set(flatten_2d(df['Комфорт'].fillna('[]').apply(ast.literal_eval).tolist())):
    df[col]=df['Комфорт'].fillna('[]').apply(ast.literal_eval).map(lambda x: col in x)
for col in set(flatten_2d(df['Интериор'].fillna('[]').apply(ast.literal_eval).tolist())):
    df[col]=df['Интериор'].fillna('[]').apply(ast.literal_eval).map(lambda x: col in x)
for col in set(flatten_2d(df['Безопасност'].fillna('[]').apply(ast.literal_eval).tolist())):
    df[col]=df['Безопасност'].fillna('[]').apply(ast.literal_eval).map(lambda x: col in x)
for col in set(flatten_2d(df['Други'].fillna('[]').apply(ast.literal_eval).tolist())):
    df[col]=df['Други'].fillna('[]').apply(ast.literal_eval).map(lambda x: col in x)
for col in set(flatten_2d(df['Екстериор'].fillna('[]').apply(ast.literal_eval).tolist())):
    df[col]=df['Екстериор'].fillna('[]').apply(ast.literal_eval).map(lambda x: col in x)
for col in set(flatten_2d(df['Защита'].fillna('[]').apply(ast.literal_eval).tolist())):
    df[col]=df['Защита'].fillna('[]').apply(ast.literal_eval).map(lambda x: col in x)

In [ ]:
df.columns
df=df.drop(columns=['ДвигателInfo', 'МощностInfo',
       'Скоростна кутияInfo', 'Пробег [км]Info',
       'ЕвростандартInfo','CarModel', 'CarDescription',
       'Location','Дата на производствоTech','Комфорт','Интериор'
       ,'Безопасност','Други','Екстериор','Защита'])
df=df.rename(columns={'ДвигателTech':'Двигател','МощностTech':'Мощност',
                           'Скоростна кутияTech':'Скоростна кутия',
                           'Пробег [км]Tech':'Пробег [км]',
                           'ЕвростандартTech':'Евростандарт'})

In [ ]:
df.columns

In [ ]:
df.to_csv('out_cleared.csv')

In [ ]:
df.loc[df['CarPrice']==484900.0,'CarPrice']=48900.0


In [ ]:
df.dropna(subset=['CarPrice']).sort_values('CarPrice',ascending=False).loc[:,['Brand','Model','CarPrice','URL','Мощност','Пробег [км]','Цвят','Двигател']].head(20)


In [ ]:
brand_to_price=df.groupby('Brand')[['CarPrice','Мощност']].agg(['mean','count','max','min','median'])
plt.rcParams["figure.figsize"] = (20,10)
brand_to_price['CarPrice'][brand_to_price['CarPrice']['count']>=20].plot.bar(y=['mean','median'])

In [ ]:
brand_to_price['Мощност'][brand_to_price['Мощност']['count']>=20].plot.bar(y=['mean','median'])

In [ ]:
brands_counts=df['Brand'].value_counts()
brands_counts['Others']=brands_counts[brands_counts<=12].sum()
brands_counts[brands_counts>12].plot.pie()

In [ ]:
df['Двигател'].value_counts().plot.pie()

In [ ]:
df.groupby('Двигател')[['CarPrice','Мощност']].agg(['mean','median']).plot.bar(subplots=True)


In [ ]:
color_to_other=df.groupby('Цвят')[['CarPrice','Мощност']].agg(['mean','median'])
color_to_other['CarPrice'].plot.bar()
color_to_other['Мощност'].plot.bar()

In [ ]:
df.groupby('Скоростна кутия')['CarPrice'].mean().plot.barh()

In [ ]:
df.groupby('Категория')['CarPrice'].mean().plot.bar()

In [ ]:
df.groupby('Year')['CarPrice'].mean().plot.line()
df.groupby('Year')['CarPrice'].median().plot.line()

In [ ]:
region_to_car_price=df.groupby('Region')['CarPrice'].agg(['median','mean','count'])
region_to_car_price[region_to_car_price['count']>10].plot.bar(y='median')